<a href="https://colab.research.google.com/github/sandana123kri/Deep_learning_Pytorch/blob/main/pytorch_Rnn_based_QA_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df=pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
### tokenize

def tokenize(text):
  text=text.lower()
  text=text.replace('?','')
  text=text.replace("'","")
  return text.split()

In [3]:
tokenize("What is the capital of France")

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
# vocab
vocab ={'<UNK>':0}

In [5]:
def build_vocab(row):
  tokenized_question=tokenize(row['question'])
  tokenized_answer=tokenize(row['answer'])

  merged_tokens=tokenized_question+tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token]=len(vocab)

In [6]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [7]:
### convert text to numerical indexes

def text_to_indices(text,vocab):
  indexed_text=[]

  tokenized_text=tokenize(text)

  for token in tokenized_text:
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [8]:
import torch
from torch.utils.data import Dataset,DataLoader

In [9]:
class QAdataset(Dataset):
  def __init__(self,df,vocab):
    self.df=df
    self.vocab=vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question=text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer=text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [10]:
dataset=QAdataset(df,vocab)

In [11]:
dataloader=DataLoader(dataset,batch_size=1,shuffle=True)

In [12]:
for question, answer in dataloader:
  print(question,answer[0]);

tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([205])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([179])
tensor([[ 10,  96,   3, 104, 239]]) tensor([240])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([116])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([7])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([233])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([207])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([287])
tensor([[ 10, 308,   3, 309, 310]]) tensor([311])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([260])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[10, 96,  3, 97]]) tensor([98])
tensor([[ 10,  75, 111]]) tensor([112])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([95])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) t

In [13]:
import torch.nn as nn

In [18]:
class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embeeding=nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn=nn.RNN(50,64,batch_first=True)
    self.fc=nn.Linear(64,vocab_size)

  def forward(self,question):
    embeeded_question=self.embeeding(question)
    hidden,final=self.rnn(embeeded_question)
    output=self.fc(final.squeeze(0))
    return output

In [19]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [20]:
learning_rate = 0.001
epochs = 20

In [21]:
model = SimpleRNN(len(vocab))

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 524.036850
Epoch: 2, Loss: 450.103185
Epoch: 3, Loss: 370.949514
Epoch: 4, Loss: 313.096303
Epoch: 5, Loss: 262.108876
Epoch: 6, Loss: 214.749764
Epoch: 7, Loss: 170.964032
Epoch: 8, Loss: 133.150618
Epoch: 9, Loss: 102.193003
Epoch: 10, Loss: 78.160554
Epoch: 11, Loss: 59.760168
Epoch: 12, Loss: 46.468442
Epoch: 13, Loss: 37.125193
Epoch: 14, Loss: 29.692411
Epoch: 15, Loss: 24.439566
Epoch: 16, Loss: 20.385777
Epoch: 17, Loss: 16.901734
Epoch: 18, Loss: 14.388382
Epoch: 19, Loss: 12.357059
Epoch: 20, Loss: 10.661286


In [24]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [25]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [27]:
predict(model, "What is the capital city of france?")

paris
